<a href="https://colab.research.google.com/github/Andrey793/first_kaggle_project/blob/main/my_first_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pytorch_lightning torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [2]:
import math
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm


In [3]:
import pandas as pd
import numpy as np

In [156]:
train_dat = pd.read_csv('/content/train.csv')
test_dat = pd.read_csv('/content/test.csv')
train_dat.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [ ]:
train_dat.shape , test_dat.shape

((990, 194), (594, 193))

In [158]:
X_train = train_dat.drop(columns=['species'])
y_train = train_dat['species']
X_val = test_dat

In [144]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [145]:
categories = [[*set(np.concatenate([np.array(y_train.unique()), np.array(y_test.unique())]))]]

In [8]:
from sklearn.preprocessing import OrdinalEncoder

In [146]:
enc = OrdinalEncoder(categories=categories)
y_train = enc.fit_transform(np.array(y_train).reshape(-1, 1))
y_test = enc.transform(np.array(y_test).reshape(-1, 1))

In [147]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [52]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 7.1 MB/s eta 0:00:00


In [53]:
from catboost import CatBoostClassifier

In [150]:
model = CatBoostClassifier(iterations = 7500, loss_function='MultiClass', task_type='GPU', classes_count=99, verbose=100)

In [151]:
model.fit(np.concatenate((X_train, X_test)), np.concatenate((y_train, y_test)))

Learning rate set to 0.012091
0:	learn: 4.5787810	total: 96.7ms	remaining: 12m 4s
100:	learn: 3.3070939	total: 7.91s	remaining: 9m 39s
200:	learn: 2.3528843	total: 15.9s	remaining: 9m 38s
300:	learn: 1.6759219	total: 24.1s	remaining: 9m 36s
400:	learn: 1.2188462	total: 32.2s	remaining: 9m 30s
500:	learn: 0.8937692	total: 40.7s	remaining: 9m 28s
600:	learn: 0.6849948	total: 48.9s	remaining: 9m 21s
700:	learn: 0.5383174	total: 57.2s	remaining: 9m 15s
800:	learn: 0.4340609	total: 1m 5s	remaining: 9m 6s
900:	learn: 0.3596371	total: 1m 13s	remaining: 9m
1000:	learn: 0.3028525	total: 1m 22s	remaining: 8m 53s
1100:	learn: 0.2596969	total: 1m 30s	remaining: 8m 45s
1200:	learn: 0.2268797	total: 1m 38s	remaining: 8m 37s
1300:	learn: 0.2006188	total: 1m 47s	remaining: 8m 30s
1400:	learn: 0.1778557	total: 1m 55s	remaining: 8m 23s
1500:	learn: 0.1600178	total: 2m 3s	remaining: 8m 15s
1600:	learn: 0.1455737	total: 2m 12s	remaining: 8m 7s
1700:	learn: 0.1328537	total: 2m 20s	remaining: 7m 58s
1800:	l

In [10]:
from torchvision.transforms import v2

In [ ]:
#разместил классы по папкам для ImageFolder

In [11]:
for i in range(X_train.shape[0]):
  new_path = '/content/images_classified/train/class' + str(int(y_train[i][0]))
  old_path = '/content/images/'
  if not os.path.isdir(new_path):
    os.mkdir(new_path)
  os.rename(old_path + str(np.array(X_train['id'])[i]) + '.jpg', new_path + '/' + str(np.array(X_train['id'])[i]) + '.jpg')

In [12]:
for i in range(X_test.shape[0]):
  new_path = '/content/images_classified/test/class' + str(int(y_test[i][0]))
  old_path = '/content/images/'
  if not os.path.isdir(new_path):
    os.mkdir(new_path)
  os.rename(old_path + str(np.array(X_test['id'])[i]) + '.jpg', new_path + '/' + str(np.array(X_test['id'])[i]) + '.jpg')

In [27]:
preproc = v2.Compose([
    v2.PILToTensor(),
    v2.Resize(size=(224, 224)),
    v2.ToDtype(torch.float32, scale=True),  # to float32 in [0, 1]
])


train_dataset = ImageFolder('/content/images_classified/train', transform=preproc)
val_dataset = ImageFolder('/content/images_classified/test', transform=preproc)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [14]:
NUM_CLASSES = 99

In [15]:
from torchvision.models import resnet18
from torchmetrics.functional import accuracy
device = 'cuda:0'

In [29]:
class YourModule(pl.LightningModule):
    def __init__(self, learning_rate=0.001, pretrained: bool = False) -> None:
        super().__init__()
        self.acc = 0
        self.pretrained = pretrained
        self.learning_rate = learning_rate
        self.add_l4 = False
        self.add_l3 = False
        self.add_l2 = False
        if pretrained:
            self.model = resnet18(weights='DEFAULT').to(device)
            for param in self.model.parameters():
              param.requires_grad = False
            self.model.fc = nn.Linear(self.model.fc.in_features, NUM_CLASSES)
            self.optimizer = torch.optim.Adam(self.model.fc.parameters(), lr=0.0001)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x) -> torch.Tensor:
        if self.pretrained:
            if self.acc > 0.85 and not self.add_l4:
              for param in self.model.layer4.parameters():
                param.requires_grad = True
              self.optimizer.add_param_group({'params' : list(self.model.layer4.parameters())})
              self.add_l4 = True
            if self.acc > 0.915 and not self.add_l3:
              for param in self.model.layer3.parameters():
                param.requires_grad = True
              self.optimizer.add_param_group({'params' : list(self.model.layer3.parameters())})
              self.add_l3 = True
            if self.acc > 0.94 and not self.add_l2:
              for param in self.model.layer2.parameters():
                param.requires_grad = True
              self.optimizer.add_param_group({'params' : list(self.model.layer2.parameters())})
              self.add_l2 = True
            preds = self.model(x)
        return preds

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        self.model.train()
        images, target = train_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx) -> None:
        self.model.eval()
        images, target = val_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        self.acc = accuracy(torch.argmax(preds, dim=-1).long(), target.long(), task='multiclass', num_classes=NUM_CLASSES)
        self.log("val_loss", loss, prog_bar=True)
        self.log("accuracy", self.acc, prog_bar=True)

In [30]:
module = YourModule(pretrained=True)
module.model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [31]:
trainer = pl.Trainer( max_epochs=60, log_every_n_steps=10)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [32]:
trainer.fit(module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | ResNet           | 11.2 M | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
50.8 K    Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.909    Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=60` reached.


In [ ]:
#Добавляю в текстовые данные предсказания по картинкам

In [33]:
module.model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [39]:
extra_train = np.zeros((X_train.shape[0], 2))
for i in range(X_train.shape[0]):
  extra_train[i][0] = np.array(X_train['id'])[i]
  path = '/content/images2/' + str(np.array(X_train['id'])[i]) + '.jpg'
  img = Image.open(path).convert('RGB')
  img = preproc(img)
  img.unsqueeze_(0)
  extra_train[i][1] = torch.argmax(module.model(img).data)

In [40]:
extra_train = pd.DataFrame(extra_train)
extra_train[0] = extra_train[0].astype(int)
extra_train.rename(columns={0 : 'id'}, inplace=True)
extra_train

,id,1
0,1042,67.0
1,789,94.0
2,1268,47.0
3,900,87.0
4,280,1.0
...,...,...
737,823,84.0
738,1041,13.0
739,898,31.0
740,334,44.0


In [42]:
X_train_full = pd.merge(X_train, extra_train, on='id')

In [43]:
extra_test = np.zeros((X_test.shape[0], 2))
for i in range(X_test.shape[0]):
  extra_test[i][0] = np.array(X_test['id'])[i]
  path = '/content/images2/' + str(np.array(X_test['id'])[i]) + '.jpg'
  img = Image.open(path).convert('RGB')
  img = preproc(img)
  img.unsqueeze_(0)
  extra_test[i][1] = torch.argmax(module.model(img).data)

In [44]:
extra_test = pd.DataFrame(extra_test)
extra_test[0] = extra_test[0].astype(int)
extra_test.rename(columns={0 : 'id'}, inplace=True)
extra_test

,id,1
0,461,41.0
1,1231,68.0
2,331,36.0
3,398,33.0
4,865,73.0
...,...,...
243,1163,53.0
244,80,69.0
245,1502,39.0
246,87,20.0


In [45]:
X_test_full = pd.merge(X_test, extra_test, on='id')

In [46]:
X_test_full

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,1
0,461,0.000000,0.001953,0.027344,0.013672,0.021484,0.007812,0.009766,0.0,0.017578,...,0.000000,0.000000,0.051758,0.001953,0.034180,0.0,0.043945,0.010742,0.000000,41.0
1,1231,0.001953,0.003906,0.046875,0.003906,0.017578,0.005859,0.027344,0.0,0.003906,...,0.000000,0.032227,0.000000,0.020508,0.000000,0.0,0.000000,0.004883,0.001953,68.0
2,331,0.007812,0.019531,0.083984,0.015625,0.001953,0.017578,0.000000,0.0,0.050781,...,0.042969,0.002930,0.002930,0.021484,0.000000,0.0,0.027344,0.000000,0.008789,36.0
3,398,0.056641,0.091797,0.007812,0.005859,0.001953,0.091797,0.076172,0.0,0.000000,...,0.000000,0.002930,0.000977,0.028320,0.000000,0.0,0.001953,0.000000,0.013672,33.0
4,865,0.003906,0.001953,0.021484,0.001953,0.013672,0.000000,0.000000,0.0,0.011719,...,0.000000,0.000000,0.007812,0.007812,0.000000,0.0,0.048828,0.008789,0.006836,73.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,1163,0.041016,0.074219,0.029297,0.021484,0.005859,0.066406,0.017578,0.0,0.003906,...,0.003906,0.003906,0.033203,0.022461,0.001953,0.0,0.049805,0.005859,0.001953,53.0
244,80,0.000000,0.001953,0.001953,0.005859,0.035156,0.000000,0.003906,0.0,0.013672,...,0.000000,0.012695,0.001953,0.033203,0.000000,0.0,0.000000,0.022461,0.025391,69.0
245,1502,0.001953,0.003906,0.015625,0.025391,0.005859,0.000000,0.013672,0.0,0.005859,...,0.000000,0.003906,0.000977,0.021484,0.000000,0.0,0.021484,0.003906,0.043945,39.0
246,87,0.001953,0.001953,0.093750,0.017578,0.009766,0.046875,0.013672,0.0,0.005859,...,0.000000,0.002930,0.072266,0.007812,0.100590,0.0,0.058594,0.005859,0.000000,20.0


In [47]:
X_train_full = X_train_full.drop(columns='id')
X_test_full.drop(columns='id', inplace=True)

In [48]:
X_train_full.rename(str, axis='columns', inplace=True)
X_test_full.rename(str, axis='columns', inplace=True)

In [49]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_full = scaler.fit_transform(X_train_full)
X_test_full = scaler.transform(X_test_full)

In [54]:
model = CatBoostClassifier(iterations = 10000, loss_function='MultiClass', od_type='IncToDec', use_best_model=True, task_type='GPU', classes_count=99, verbose=100)

In [55]:
model.fit(X_train_full, y_train, eval_set=(X_test_full, y_test))

Learning rate set to 0.037303
0:	learn: 4.5543529	test: 4.5621328	best: 4.5621328 (0)	total: 99.9ms	remaining: 16m 39s
100:	learn: 1.6750750	test: 2.0511097	best: 2.0511097 (100)	total: 7.07s	remaining: 11m 32s
200:	learn: 0.6631962	test: 1.0775196	best: 1.0775196 (200)	total: 14.3s	remaining: 11m 37s
300:	learn: 0.3389287	test: 0.7036052	best: 0.7036052 (300)	total: 21.5s	remaining: 11m 34s
400:	learn: 0.2162823	test: 0.5375030	best: 0.5375030 (400)	total: 28.4s	remaining: 11m 20s
500:	learn: 0.1534094	test: 0.4427667	best: 0.4427667 (500)	total: 35.4s	remaining: 11m 10s
600:	learn: 0.1164948	test: 0.3812304	best: 0.3812304 (600)	total: 42.3s	remaining: 11m 1s
700:	learn: 0.0922610	test: 0.3362507	best: 0.3362507 (700)	total: 49.3s	remaining: 10m 54s
800:	learn: 0.0758587	test: 0.3036469	best: 0.3036469 (800)	total: 56.4s	remaining: 10m 48s
900:	learn: 0.0646444	test: 0.2802333	best: 0.2802333 (900)	total: 1m 3s	remaining: 10m 39s
1000:	learn: 0.0560249	test: 0.2612169	best: 0.2612169

In [103]:
model = CatBoostClassifier(iterations = 10000, loss_function='MultiClass',task_type='GPU', classes_count=99, verbose=100)

In [104]:
model.fit(np.concatenate((X_train_full, X_test_full)), np.concatenate((y_train, y_test)))

Learning rate set to 0.009515
0:	learn: 4.5783410	total: 172ms	remaining: 28m 38s
100:	learn: 3.3615446	total: 7.95s	remaining: 12m 59s
200:	learn: 2.4380951	total: 15.6s	remaining: 12m 41s
300:	learn: 1.7382709	total: 23.5s	remaining: 12m 35s
400:	learn: 1.2610754	total: 31.5s	remaining: 12m 34s
500:	learn: 0.9339041	total: 39.7s	remaining: 12m 33s
600:	learn: 0.7133737	total: 47.9s	remaining: 12m 29s
700:	learn: 0.5598490	total: 56s	remaining: 12m 22s
800:	learn: 0.4579271	total: 1m 3s	remaining: 12m 12s
900:	learn: 0.3813071	total: 1m 11s	remaining: 12m 2s
1000:	learn: 0.3247709	total: 1m 19s	remaining: 11m 52s
1100:	learn: 0.2801439	total: 1m 27s	remaining: 11m 44s
1200:	learn: 0.2467357	total: 1m 34s	remaining: 11m 34s
1300:	learn: 0.2210449	total: 1m 42s	remaining: 11m 24s
1400:	learn: 0.1988528	total: 1m 50s	remaining: 11m 15s
1500:	learn: 0.1813366	total: 1m 57s	remaining: 11m 5s
1600:	learn: 0.1652300	total: 2m 5s	remaining: 10m 57s
1700:	learn: 0.1521532	total: 2m 12s	remaini

In [105]:
extra_val = np.zeros((X_val.shape[0], 2))
for i in range(X_val.shape[0]):
  extra_val[i][0] = np.array(X_val['id'])[i]
  path = '/content/images2/' + str(np.array(X_train['id'])[i]) + '.jpg'
  img = Image.open(path).convert('RGB')
  img = preproc(img)
  img.unsqueeze_(0)
  extra_val[i][1] = torch.argmax(module.model(img).data)

In [106]:
extra_val = pd.DataFrame(extra_val)
extra_val[0] = extra_val[0].astype(int)
extra_val.rename(columns={0 : 'id'}, inplace=True)
X_val_full = pd.merge(X_val, extra_val, on='id')

In [ ]:
#Заполняю ответы

In [107]:
submission = np.zeros((X_val_full.shape[0], NUM_CLASSES + 1))
for i in range(X_val_full.shape[0]):
  submission[i][0] = np.array(X_val_full['id'])[i]

In [108]:
X_val_full.drop(columns='id', inplace=True)
X_val_full.rename(str, axis='columns', inplace=True)
X_val_full = scaler.transform(X_val_full)

In [109]:
for i in range(X_val_full.shape[0]):
  pred = model.predict_proba(X_val_full[i])
  submission[i][1:] = model.predict_proba(X_val_full[i])

In [96]:
sample = pd.read_csv('/content/sample_submission.csv')
columns = sample.columns

In [111]:
submission = pd.DataFrame(submission, columns=columns)

In [112]:
submission['id'] = submission['id'].astype(int)

In [113]:
submission

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,0.005615,0.000899,0.001032,0.007249,0.001774,0.012101,0.002013,0.011291,0.006349,...,0.003221,0.001517,0.001197,0.005704,0.005330,0.001725,0.003535,0.007326,0.001782,0.013046
1,7,0.001151,0.001150,0.000508,0.000739,0.000809,0.520298,0.002531,0.007405,0.000809,...,0.006858,0.056653,0.002403,0.012586,0.002563,0.059966,0.002599,0.014544,0.010484,0.014765
2,9,0.000475,0.000140,0.000123,0.001115,0.000566,0.001131,0.000224,0.000244,0.000230,...,0.000334,0.000386,0.000542,0.000298,0.000629,0.000166,0.001281,0.000693,0.000206,0.000226
3,12,0.000452,0.000371,0.000272,0.000503,0.000684,0.002278,0.000725,0.000983,0.002840,...,0.001817,0.699191,0.008823,0.002875,0.002610,0.001940,0.025049,0.018920,0.003774,0.001738
4,13,0.032751,0.004126,0.006462,0.002494,0.001928,0.001834,0.000887,0.000694,0.003589,...,0.000779,0.001030,0.001110,0.001235,0.001160,0.000664,0.002226,0.001634,0.000997,0.000960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,1576,0.000195,0.000118,0.000098,0.000418,0.000266,0.000560,0.000213,0.000302,0.000808,...,0.001049,0.001305,0.002011,0.000896,0.002817,0.000401,0.010120,0.005129,0.000474,0.000665
590,1577,0.006240,0.002536,0.001899,0.002671,0.002545,0.001324,0.000495,0.000437,0.001056,...,0.000282,0.000575,0.000606,0.000495,0.000418,0.000291,0.000784,0.000641,0.000358,0.000371
591,1579,0.001555,0.000788,0.003849,0.002202,0.001161,0.003837,0.001158,0.001744,0.003947,...,0.000748,0.001045,0.001547,0.017209,0.001374,0.001010,0.001589,0.003644,0.001831,0.010074
592,1580,0.003206,0.009804,0.002192,0.002197,0.003096,0.004267,0.006672,0.005011,0.000398,...,0.000843,0.000860,0.000688,0.000357,0.000501,0.000674,0.000380,0.000379,0.000851,0.000419


In [114]:
submission.to_csv('/content/my_submission.csv', index=False)